In [1]:
pip install pytube langchain llama-index qdrant-client sentence-transformers gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150

In [2]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.0 MB/s eta 0:00:00


In [3]:
from pytube import YouTube
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant
from langchain.chains import VectorDBQA
from qdrant_client import QdrantClient
from langchain.llms import HuggingFaceHub
import gradio as gr

# Initialize Qdrant Client
qdrant_client = QdrantClient(url="http://localhost:6333")  # or your Qdrant Cloud URL
collection_name = "youtube_transcripts"

# Function to get YouTube transcript
def get_youtube_transcript(url):
    yt = YouTube(url)
    captions = yt.captions.get_by_language_code('en')
    if captions:
        return captions.generate_srt_captions()
    else:
        return None

# Function to set up vector store with Qdrant
def setup_qdrant_store(transcript):
    sentences = transcript.split('\n\n')
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Create documents
    documents = [{"text": sentence, "metadata": {"source": "YouTube"}} for sentence in sentences]

    # Store embeddings in Qdrant
    vector_store = Qdrant.from_documents(
        documents, embeddings, qdrant_client=qdrant_client, collection_name=collection_name
    )

    return vector_store

# QA System using LangChain and Hugging Face Model
def qa_system(youtube_url, question):
    transcript = get_youtube_transcript(youtube_url)
    if not transcript:
        return "Transcript not available."

    vector_store = setup_qdrant_store(transcript)
    llm = HuggingFaceHub(repo_id="facebook/bart-large-cnn")

    # Create QA chain
    qa_chain = VectorDBQA(llm=llm, vectorstore=vector_store)
    result = qa_chain.run(question)

    return result

# Gradio Interface
iface = gr.Interface(fn=qa_system, inputs=["text", "text"], outputs="text")
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b69a04c7cda6e49e66.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
